In [45]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torchinfo import summary
import matplotlib.pyplot as plt
import torch.optim as optim
import torchmetrics
import numpy as np
import random

In [46]:
# Set the seed
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)

In [47]:
# mean and sd for images size 299,299
mean = [0.1854, 0.1854, 0.1855]
std = [0.2005, 0.2005, 0.2005]

train_ratio = 0.8 # train, valid split
batch_size = 32
epochs = 10
lr = 0.0001

# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [48]:
data_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [49]:
# Load the datasets
train_dataset = datasets.ImageFolder(root="../data/Training", transform=data_transform)
test_dataset = datasets.ImageFolder(root="../data/Testing", transform=data_transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Split Train into Train, Valid
train_size = int(train_ratio * len(train_dataset))
valid_size = len(train_dataset) - train_size
train_subset, valid_subset = random_split(train_dataset, [train_size, valid_size])

# Create DataLoaders
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(valid_subset, batch_size=batch_size, shuffle=False, num_workers=2)

In [50]:
# Initialize Inception V3 Model
model = models.inception_v3(weights=True)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, len(train_dataset.classes))
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

c:\Users\jesdi\.conda\envs\ml\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [51]:
# summary(model, input_size=(1, 3, 299, 299))

In [59]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    
    for epoch in range(num_epochs):
        model.train()
        loss = 0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Clear previous gradients
            optimizer.zero_grad()
            
            # Inception v3 has two outputs: primary and auxiliary
            outputs, aux_outputs = model(images)  
            # calculate weighted loss
            loss1 = criterion(outputs, labels)
            loss2 = criterion(aux_outputs, labels)
            _loss = loss1 + 0.4 * loss2
            
            # Backpropagate and update weights
            _loss.backward()  
            optimizer.step()
            
            loss += _loss.item()
            
            # Get predicted class
            _, predicted = outputs.max(1)  
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
        epoch_loss = loss / len(train_loader)
        accuracy = 100 * correct / total
        
        # Validation Phase
        model.eval()
        loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _loss = criterion(outputs, labels)
                
                loss += _loss.item()
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
        val_epoch_loss = loss / len(val_loader)
        val_accuracy = 100 * correct / total
        
        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f} - Train Acc: {accuracy:.2f} - Val Loss: {val_epoch_loss:.4f} - Val Accuracy: {val_accuracy:.2f}%")

In [60]:
# Train the model
epochs = 2
train_model(model, train_loader, criterion, optimizer, num_epochs=epochs)

Epoch 1/2 - Train Loss: 0.0228 - Train Acc: 99.52 - Val Loss: 0.0478 - Val Accuracy: 98.34%
Epoch 2/2 - Train Loss: 0.0206 - Train Acc: 99.47 - Val Loss: 0.1561 - Val Accuracy: 96.59%
